In [175]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


In [176]:
data=pd.read_csv("../merged/len3_ndb_agg_blp_DropNever.csv",index_col=0,encoding="shift-jis")
data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,id_399.0,id_441.0,id_449.0,id_625.0,id_629.0,id_799.0,q_share,r_share,generic_share_q,generic_share_r
0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,2.489539e+07,2014,1,...,0,0,0,0,0,0,0.007258,0.002068,0.236142,0.053889
1,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,1.024590e+07,2014,1,...,0,0,0,0,0,0,0.002987,0.001507,0.236142,0.053889
2,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,5.025912e+07,2014,0,...,0,0,0,0,0,0,0.014653,0.004175,0.236142,0.053889
3,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,2.742935e+07,2014,0,...,0,0,0,0,0,0,0.007997,0.004036,0.236142,0.053889
4,114.0,解熱鎮痛消炎剤,661140081.0,ボルタレンサポ５０ｍｇ,1147700J3084,63.1,0.0,5.339043e+06,2014,1,...,0,0,0,0,0,0,0.001557,0.002136,0.236142,0.053889


In [177]:
data=data.drop_duplicates(subset=["薬効分類","year"])
# drop never treated group
data.head(),data.shape

(      薬効分類   薬効分類名称       医薬品コード          医薬品名  薬価基準収載医薬品コード    薬価  後発品区分  \
 0    114.0  解熱鎮痛消炎剤  620007096.0    ボルタレン錠２５ｍｇ  1147002F1560  13.1    0.0   
 80   121.0    局所麻酔剤  620005991.0    ストロカイン錠５ｍｇ  1219002F1065   5.7    0.0   
 96   123.0    自律神経剤  620002526.0  チアトンカプセル１０ｍｇ  1231013M2179  15.2    0.0   
 147  131.0     眼科用剤  620207901.0   アダプチノール錠５ｍｇ  1319004F1035  45.9    0.0   
 199  132.0    耳鼻科用剤  620004822.0    プリビナ液０．０５％  1324704Q1033   4.5    0.0   
 
                総計  year  in_hospital  ...  id_399.0  id_441.0  id_449.0  \
 0    2.489539e+07  2014            1  ...         0         0         0   
 80   1.161969e+07  2014            1  ...         0         0         0   
 96   9.128303e+06  2014            1  ...         0         0         0   
 147  5.229358e+06  2014            1  ...         0         0         0   
 199  6.506744e+06  2014            1  ...         0         0         0   
 
     id_625.0 id_629.0 id_799.0   q_share   r_share generic_share_q  \
 

In [178]:
data_map=pd.read_csv("../generic/generic_usage_imp.csv")
generic_per_map={}
for i in range(data_map.shape[0]):
    generic_per_map[data_map.iloc[i,0]]=data_map.iloc[i,1]
generic_per_map[2023]=83
generic_per_map

{2007: 34.9,
 2008: 35.35,
 2009: 35.8,
 2010: 37.85,
 2011: 39.9,
 2012: 43.4,
 2013: 46.9,
 2014: 51.55,
 2015: 56.2,
 2016: 61.0,
 2017: 65.8,
 2018: 72.6,
 2019: 76.7,
 2020: 78.3,
 2021: 79.0,
 2022: 79.0,
 2023: 83}

In [179]:
# set(data["薬効分類"])
data.fillna(0,inplace=True)
data[data["generic_share_q"].isnull()]

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,id_399.0,id_441.0,id_449.0,id_625.0,id_629.0,id_799.0,q_share,r_share,generic_share_q,generic_share_r


In [180]:
data=data.loc[:,["薬効分類","year","generic_per",'generic_share_q', 'generic_share_r']]
final_cols=["薬効分類","year",'generic_share_q', 'generic_share_r']
final_cols

['薬効分類', 'year', 'generic_share_q', 'generic_share_r']

In [181]:
# 33 ids exist
add_df=pd.DataFrame(columns=data.columns)
for id in set(data["薬効分類"]):
    for year in range(2008,2024):
        if year not in data[data["薬効分類"]==id]["year"].values:
            # 229 の2018までと224の2017以降は0
            if id==229 and year<=2018:
                add_df=add_df.append(pd.DataFrame([[id,year,generic_per_map[year]]+[0.0]*(data.shape[1]-3)],columns=data.columns),ignore_index=True)
            elif id==224 and year>=2017:
                add_df=add_df.append(pd.DataFrame([[id,year,generic_per_map[year]]+[0.0]*(data.shape[1]-3)],columns=data.columns),ignore_index=True)
            else:
                add_df=add_df.append(pd.DataFrame([[id,year,generic_per_map[year]]+[np.nan]*(data.shape[1]-3)],columns=data.columns),ignore_index=True)
add_df.sort_values(by=["薬効分類","year"],inplace=True)
add_df.reset_index(drop=True,inplace=True)
add_df,add_df.shape

C:\Users\hakos\AppData\Local\Temp\ipykernel_10196\1021681621.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  add_df=add_df.append(pd.DataFrame([[id,year,generic_per_map[year]]+[np.nan]*(data.shape[1]-3)],columns=data.columns),ignore_index=True)
C:\Users\hakos\AppData\Local\Temp\ipykernel_10196\1021681621.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  add_df=add_df.append(pd.DataFrame([[id,year,generic_per_map[year]]+[0.0]*(data.shape[1]-3)],columns=data.columns),ignore_index=True)


(      薬効分類  year generic_per generic_share_q generic_share_r
 0    114.0  2008       35.35             NaN             NaN
 1    114.0  2009        35.8             NaN             NaN
 2    114.0  2010       37.85             NaN             NaN
 3    114.0  2011        39.9             NaN             NaN
 4    114.0  2012        43.4             NaN             NaN
 ..     ...   ...         ...             ...             ...
 195  799.0  2011        39.9             NaN             NaN
 196  799.0  2012        43.4             NaN             NaN
 197  799.0  2013        46.9             NaN             NaN
 198  799.0  2022        79.0             NaN             NaN
 199  799.0  2023          83             NaN             NaN
 
 [200 rows x 5 columns],
 (200, 5))

In [182]:
dummies=pd.get_dummies(data["薬効分類"])
data=pd.concat([data,dummies],axis=1)
data

,薬効分類,year,generic_per,generic_share_q,generic_share_r,114.0,121.0,123.0,131.0,132.0,...,264.0,265.0,332.0,339.0,399.0,441.0,449.0,625.0,629.0,799.0
0,114.0,2014,51.55,0.236142,0.053889,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80,121.0,2014,51.55,0.062114,0.053451,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,123.0,2014,51.55,0.129137,0.064243,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
147,131.0,2014,51.55,0.136869,0.080471,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
199,132.0,2014,51.55,0.026413,0.058417,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25142,441.0,2021,79.00,0.342768,0.263456,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
25182,449.0,2021,79.00,0.608051,0.363450,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
25386,625.0,2021,79.00,0.502607,0.052158,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
25628,629.0,2021,79.00,0.284349,0.073842,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [183]:
dummies=pd.get_dummies(add_df["薬効分類"])
add_df=pd.concat([add_df,dummies],axis=1)
add_df

c:\Users\hakos\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


,薬効分類,year,generic_per,generic_share_q,generic_share_r,114.0,121.0,123.0,131.0,132.0,...,264.0,265.0,332.0,339.0,399.0,441.0,449.0,625.0,629.0,799.0
0,114.0,2008,35.35,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,114.0,2009,35.8,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,114.0,2010,37.85,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,114.0,2011,39.9,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,114.0,2012,43.4,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,799.0,2011,39.9,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
196,799.0,2012,43.4,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
197,799.0,2013,46.9,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
198,799.0,2022,79.0,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [184]:
# data_est
# ids=data.iloc[:,4:]
# X=data[["year"]+list(ids.columns)]
# y=data["generic_share_q"]
# # # X = sm.add_constant(X)  # 定数項を追加
# # model = sm.OLS(y, X).fit()
# X


In [185]:
data=data.append(add_df,ignore_index=True)
data.sort_values(by=["薬効分類","year"],inplace=True)
data.reset_index(drop=True,inplace=True)
data

C:\Users\hakos\AppData\Local\Temp\ipykernel_10196\1296814082.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(add_df,ignore_index=True)


,薬効分類,year,generic_per,generic_share_q,generic_share_r,114.0,121.0,123.0,131.0,132.0,...,264.0,265.0,332.0,339.0,399.0,441.0,449.0,625.0,629.0,799.0
0,114.0,2008,35.35,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,114.0,2009,35.8,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,114.0,2010,37.85,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,114.0,2011,39.9,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,114.0,2012,43.4,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,799.0,2019,76.7,0.006196,0.024777,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
396,799.0,2020,78.3,0.006056,0.022847,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
397,799.0,2021,79.0,0.007953,0.028184,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
398,799.0,2022,79.0,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [186]:
from sklearn.metrics import mean_absolute_error
from sklearn.base import BaseEstimator, RegressorMixin

class SMWrapper(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels regressors """
    def __init__(self, model_class, fit_intercept=True):
        self.model_class = model_class
        self.fit_intercept = fit_intercept
    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)
        self.model_ = self.model_class(y, X)
        self.results_ = self.model_.fit(disp=0)
        return self
    def predict(self, X):
        if self.fit_intercept:
            X = sm.add_constant(X)
        return self.results_.predict(X)


In [187]:
from sklearn.metrics import mean_absolute_error
# from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor
def custom_scorer(model, X, y):
    y_pred = model.predict(X)
    # return -np.sqrt(mean_squared_error(y, y_pred))
    # return -np.sqrt(mean_squared_error(y, y_pred))
    return -mean_absolute_error(y, y_pred)
data.columns=data.columns.astype(str)
# data["year_sq"]=data["year"]**2
# data["generic_per_sq"]=data["generic_per"]**2
for target in ["generic_share_q","generic_share_r"]:
    data_est=data.dropna(subset=[target])
    ids=data_est.iloc[:,5:]
    # x_cols=["year","year_sq","generic_per","generic_per_sq"]+list(ids.columns)
    x_cols=["year","generic_per"]+list(ids.columns)
    x_cols=np.array(x_cols).astype(str)
    X=data_est.loc[:,x_cols]
    y=data_est[target] 

    # test train split
    x_cols=np.array(x_cols).astype(str)
    X_train, X_test, y_train, y_test = train_test_split(data_est.loc[:,x_cols], data_est[target], test_size=0.2, random_state=42)
    # random forest ver
    model = RandomForestRegressor()
    cv_score=cross_val_score(model,X_train,y_train,cv=5,scoring=custom_scorer)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Calculate accuracy
    MSE = mean_squared_error(y_test, y_pred)
    MAE = mean_absolute_error(y_test, y_pred)
    print('RF MSE:', MSE,"MAE:",MAE,"CV:",cv_score.mean())

    # logit ver
    logit_model=sm.Logit(y_train.astype(float),X_train.astype(float))
    result=logit_model.fit(method="newton",disp=0,maxiter=1000)
    cv_score=cross_val_score(SMWrapper(sm.Logit),X_train.astype(float),y_train.astype(float),cv=5,scoring=custom_scorer)
    y_pred2=result.predict(X_test.astype(float))
    # calculate metrics
    MSE = mean_squared_error(y_test, y_pred2)
    MAE = mean_absolute_error(y_test, y_pred2)
    print('LG MSE:', MSE,"MAE:",MAE,"CV:",cv_score.mean())
    
    # impute missing values
    missing=data[data[target].isnull()]
    missing=missing[x_cols].astype(float)
    # missing["const"]=1
    predicted_values=model.predict(missing)
    # print(sum(predicted_values<0))
    missing[target]=predicted_values
    # update data
    data.update(missing)

    # # plot metrics
    # plt.rcParams['font.family'] = 'MS Gothic'
    # # 実際の値と予測値の散布図をプロット
    # plt.scatter(y_test, y_pred)
    # plt.scatter(y_test, y_pred2,color="m")
    # plt.plot(y_test, y_test, color='red')
    # plt.xlabel("実際の値")
    # plt.ylabel("予測値")
    # plt.title("実際の値 vs. 予測値")
    # plt.show()
    # residuals = y_test - y_pred
    # residuals2 = y_test - y_pred2
    # plt.scatter(y_test, residuals)
    # plt.scatter(y_test, residuals2,color="m")
    # plt.xlabel("実際の値")
    # plt.ylabel("残差")
    # plt.axhline(y=0, color='r', linestyle='-')  # 残差が0のラインを引く
    # plt.title("実際の値 vs. 残差")
    # plt.show()
    


RF MSE: 0.003011224600197309 MAE: 0.043354859450686956 CV: -0.06322306696985658
LG MSE: 0.00357723942116724 MAE: 0.04937514845817929 CV: -0.0518985976771443
RF MSE: 0.0011232808190678341 MAE: 0.020994136286189628 CV: -0.03666686915267588
LG MSE: 0.0018502053746161555 MAE: 0.03174587383574852 CV: -0.03863073640511635


In [188]:
data=data[final_cols]

In [189]:
data.to_csv("../merged/len3_ndb_generic_imputed.csv",encoding="shift-jis")